In [1]:
# =============================================
# 1. Imports and Setup
# =============================================
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# =============================================
# 2. Load and Preprocess the IMDB Dataset
# =============================================
num_words = 10000    # Only the top 10,000 most frequent words will be considered
maxlen = 500         # Pad/truncate all reviews to 500 words

# Load the IMDB dataset (reviews are already integer encoded)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Limit the dataset size for faster training:
# For example, take only the first 3000 training samples and the first 1000 test samples.
x_train, y_train = x_train[:100], y_train[:100]
x_test, y_test = x_test[:20], y_test[:20]


# Pad sequences so all have the same length (500 words)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test  = pad_sequences(x_test, maxlen=maxlen)

# =============================================
# 3. Split the Training Data into Training and Validation Sets
# =============================================
# We'll reserve 20% of the training data for validation.
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# =============================================
# 4. Build the RNN Model Using SimpleRNN
# =============================================
model = Sequential([
    # Embedding layer: Converts word indices to 32-dimensional vectors.
    Embedding(input_dim=num_words, output_dim=32, input_length=maxlen),
    
    # SimpleRNN layer with 32 units processes the sequence data.
    SimpleRNN(32),
    
    # Dropout to reduce overfitting: 20% of inputs will be zeroed out randomly.
    Dropout(0.2),
    
    # Output Dense layer: 1 neuron with sigmoid activation for binary classification.
    Dense(1, activation='sigmoid')
])

# Display the model summary
model.summary()

# =============================================
# 5. Compile the Model
# =============================================
# Use the Adam optimizer with a learning rate of 0.001.
# The loss used is BinaryCrossentropy (works with sigmoid output).
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

# =============================================
# 6. Set Up Callbacks
# =============================================
# EarlyStopping stops training if the validation loss doesn't improve for 3 epochs.
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# ModelCheckpoint saves the best model (based on lowest validation loss).
checkpoint = ModelCheckpoint(filepath='/Users/sameerkhan/Desktop/sameerkhanAIData/NLP/text_rnn_model.keras', monitor='val_loss', save_best_only=True)

# =============================================
# 7. Train the Model
# =============================================
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=3,
    validation_data=(x_val, y_val),
    callbacks=[early_stop, checkpoint]
)

# =============================================
# 8. Evaluate the Model on the Test Set
# =============================================
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


/Users/sameerkhan/anaconda3/envs/skhan3/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-04-16 01:28:58.864982: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-04-16 01:28:58.865015: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-16 01:28:58.865022: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-16 01:28:58.865036: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-16 01:28:58.865046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefin

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3


2025-04-16 01:28:59.278059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


2/2 ━━━━━━━━━━━━━━━━━━━━ 212s 109s/step - accuracy: 0.3750 - loss: 0.7234 - val_accuracy: 0.6500 - val_loss: 0.7022
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 221s 115s/step - accuracy: 0.7042 - loss: 0.6277 - val_accuracy: 0.5500 - val_loss: 0.7066
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 224s 116s/step - accuracy: 0.9052 - loss: 0.5614 - val_accuracy: 0.4000 - val_loss: 0.7150
1/1 - 8s - 8s/step - accuracy: 0.3500 - loss: 0.6989
Test Loss: 0.6989, Test Accuracy: 0.3500
